In [152]:
import os
import pandas as pd
import geopandas as gpd

import logging
logging.getLogger().setLevel(logging.INFO) # DEBUG, INFO or WARN
DROOT = '../1-data/'

In [138]:
# Loading this file might take a moment.
urbancenters = gpd.read_file(
    os.path.join(DROOT, '2-external', 'GHS_STAT_UCDB2015MT_GLOBE_R2019A', 'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg'))
urbancenters.head(2)

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,geometry
0,1.0,1.0,185.0,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,United States,...,397443.031445,444041.529529,NaN,NaN,missing,NaN,0.074385,0.226415,56.41,"MULTIPOLYGON (((-158.01244 21.42219, -157.9915..."
1,2.0,2.0,42.0,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,French Polynesia,...,0.000000,0.000000,NaN,NaN,missing,NaN,0.128,0.284119,NaN,"MULTIPOLYGON (((-149.56967 -17.51763, -149.508..."


In [180]:
cities = pd.read_excel(os.path.join(DROOT, '1-research', 'cities.latest.xlsx'))
cities.sort_values('priority').head(2)

,city_name,country_id,city_id,priority
22,São Paulo,BRA,1303,-1
0,s-Hertogenbosch,NLD,2194,0


In [169]:
# Initiate clean output list to add to excel.
cities_out = []

In [142]:
# Actually search cities here: 
city_name = "Reykjavik"
search = urbancenters[urbancenters.UC_NM_LST.str.contains(city_name)]
search[['ID_HDC_G0', 'UC_NM_MN', 'CTR_MN_ISO', 'AREA']]

,ID_HDC_G0,UC_NM_MN,CTR_MN_ISO,AREA
1509,1510.0,Reykjavik,ISL,82.0


In [147]:
# Pass the ID_HDC_G0 in here below:
ID_HDC_G0 = 1510.0
record = urbancenters[urbancenters.ID_HDC_G0 == ID_HDC_G0]
cities_out.append(record[['ID_HDC_G0', 'CTR_MN_ISO', 'UC_NM_MN']].iloc[0].to_list())
record

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,geometry
1509,1510.0,1.0,82.0,64.041586,-22.019693,64.162478,-21.744812,64.117289,-21.88496,Iceland,...,0.0,0.0,0.436648,7.0,available,12.0367,0.91085,0.091667,80.13,"MULTIPOLYGON (((-22.01969 64.16248, -21.93552 ..."


In [148]:
cities_new = pd.DataFrame(cities_out, columns=['city_id', 'country_id', 'city_name'])
cities_new['original_dataset'] = False
cities_new

,city_id,country_id,city_name,original_dataset
0,1510.0,ISL,Reykjavik,False
1,1510.0,ISL,Reykjavik,False


In [181]:
# Write out to file
cities_v2 = pd.concat([cities, cities_new])
cities_v2 = cities_v2.groupby('city_id').apply(lambda x: x.sort_values('priority', ascending=True).iloc[0])
cities_v2 = cities_v2.sort_values(['priority', 'country_id', 'city_name'])
cities_v2 = cities_v2.drop(columns='original_dataset')
cities_v2.to_excel('../1-data/1-research/cities.VERSION.xlsx', index=False)
cities_v2.head(2)

,city_name,country_id,city_id,priority
city_id,,,,
1303.0,São Paulo,BRA,1303.0,-1.0
2167.0,Amsterdam,NLD,2167.0,0.0


In [ ]:
search = urbancenters[urbancenters.CTR_MN_ISO == 'NLD']
search[['ID_HDC_G0', 'UC_NM_MN', 'CTR_MN_ISO', 'AREA']].sort_values('AREA', ascending=False)

,ID_HDC_G0,UC_NM_MN,CTR_MN_ISO,AREA
2122,2123.0,Rotterdam [The Hague],NLD,658.0
2166,2167.0,Amsterdam,NLD,345.0
2234,2235.0,Heerlen,NLD,129.0
2180,2181.0,Utrecht,NLD,124.0
2204,2205.0,Eindhoven,NLD,104.0
2137,2138.0,Leiden,NLD,97.0
2162,2163.0,Dordrecht,NLD,88.0
2225,2226.0,Arnhem,NLD,87.0
2182,2183.0,Tilburg,NLD,63.0
2143,2144.0,Zoetermeer,NLD,60.0
